# ZeRO и FSDP: параллелизм, шардирование и конфигурации

## __[DeepSpeed ZeRO](https://www.deepspeed.ai/docs/config-json/):__

Шардирование трёх ключевых состояний модели при обучении с data-parallel:  
- состояния оптимизатора (optimizer states)  
- градиенты (gradients)  
- параметры модели (parameters)  

Это снижает потребление памяти на каждый процесс/устройство без потери вычислительной гранулярности.

__Уровни (stage):__

| Stage | Что шардится | Комментарий |
|-------|-------------|-------------|
| 0 | ничего (традиционный data-parallel, репликация всех состояний) | обычный DDP |
| 1 | optimizer states | градиенты и параметры остаются реплицированными. Экономия памяти по сравнению с 0, небольшой наклад на коммуникации. |
| 2 | optimizer states + gradients | параметры всё ещё реплицированы. Больше экономия, больше наклад. |
| 3 | optimizer states + gradients + parameters | максимальная экономия памяти за счёт полной шардизации. Но самая большая нагрузка на коммуникации. |

__Актуальный (исправленный) DeepSpeed config по умолчанию:__

```python
zero_optimization = {
    "stage": 0,                                         # 0|1|2|3 — уровень ZeRO, 0 = обычный DDP
    "allgather_partitions": True,                       # собирать параметры после шага (True — дефолт)
    "reduce_scatter": True,                             # использовать reduce_scatter вместо allreduce
    "allgather_bucket_size": 5e8,                       # размер бакета (в элементах, не байтах), дефолт 500М
    "reduce_bucket_size": 5e8,                          # размер бакета для reduce-scatter, дефолт 500М
    "overlap_comm": False,                              # дефолт False; True перекрывает коммуникации с вычислениями
    "contiguous_gradients": True,                       # хранить градиенты в непрерывном блоке памяти
    "offload_optimizer": {                              # offload оптимизатора (актуально при stage >= 2)
        "device": "none",                               # варианты: "none" | "cpu" | "nvme"
        "pin_memory": False
    },
    "offload_param": {                                  # offload параметров (актуально при stage == 3)
        "device": "none",                               # "none" | "cpu" | "nvme"
        "pin_memory": False
    },
    "sub_group_size": 1e9,                              # лимит размера подгруппы при stage3 (байты)
    "stage3_max_live_parameters": 1e9,                  # ограничение активных параметров в памяти
    "stage3_max_reuse_distance": 1e9,                   # макс. расстояние повторного использования параметров
    "stage3_prefetch_bucket_size": 5e8,                 # размер префетч-бакета
    "stage3_param_persistence_threshold": 1e5,          # порог для параметров, не выгружаемых
    "stage3_gather_fp16_weights_on_model_save": True,   # собирать веса при сохранении
}
```
___

## [FSDP](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments.fsdp) (Fully Sharded Data Parallel)

Метод распределённого обучения, реализованный в PyTorch и интегрированный в библиотеку Accelerate / Transformers. Он позволяет шардировать параметры модели, градиенты и/или optimizer-состояния между устройствами, что значительно снижает расход памяти на GPU.

__Default FSDP config:__

```python
fsdp_config = {
    "fsdp_sharding_strategy": "FULL_SHARD",             # FULL_SHARD | SHARD_GRAD_OP | NO_SHARD | HYBRID_SHARD | HYBRID_SHARD_ZERO2
    "fsdp_offload_params": False,                       # False = параметры не выгружаются на CPU
    "fsdp_auto_wrap_policy": "TRANSFORMER_BASED_WRAP",  # TRANSFORMER_BASED_WRAP | SIZE_BASED_WRAP | NO_WRAP
    "fsdp_transformer_layer_cls_to_wrap": None,         # список классов слоёв (например ["BertLayer"]); None = автоопределение
    "fsdp_backward_prefetch_policy": "BACKWARD_POST",   # BACKWARD_PRE | BACKWARD_POST | NO_PREFETCH
    "fsdp_forward_prefetch": False,                     # False = не префетчить вперёд
    "fsdp_state_dict_type": "SHARDED_STATE_DICT",       # FULL_STATE_DICT | SHARDED_STATE_DICT | LOCAL_STATE_DICT (рекомендовано SHARDED)
    "fsdp_cpu_ram_efficient_loading": True,             # True = оптимизация загрузки RAM
    "fsdp_min_num_params": 1e8,                         # минимальное кол-во параметров для SIZE_BASED_WRAP (100 млн)
    "fsdp_sync_module_states": True,                    # синхронизация параметров при инициализации
    "fsdp_use_orig_params": True,                       # True = хранить ссылки на оригинальные параметры модели
    "fsdp_activation_checkpointing": False,             # использовать ли checkpointing активаций (экономия памяти)
}

```

[link](https://huggingface.co/docs/accelerate/en/usage_guides/fsdp?utm_source=chatgpt.com)


### Baseline_1_GPU

In [ ]:
name = 'Baseline_1_GPU'  # 21005 MiB

!CUDA_VISIBLE_DEVICES=1 accelerate launch \
  --num_processes 1 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode baseline \
    --bf16 \
    --torch-compile \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 17:34:41,692 [INFO] __main__: ============================================================
2025-11-03 17:34:41,692 [INFO] __main__: Запуск обучения в режиме: baseline
2025-11-03 17:34:41,692 [INFO] __main__: ============================================================
2025-11-03 17:34:41,692 [INFO] __main__: World size: 1
2025-11-03 17:34:41,692 [INFO] __main__: CUDA_VISIBLE_DEVICES: 1
2025-11-03 17:34:41,692 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 17:34:41,692 [INFO] __main__: Создание baseline setup (single GPU)
Training samples:   1940063
Validation samples: 5000
2025-11-03 17:35:16,757 [INFO] lib.modeling: Модель: 960,881,664 параметров, dtype=torch.bfloat16, ~1.

### 1_GPU_v2

In [ ]:
name = '1_GPU_v2'  # 34353 MiB

!CUDA_VISIBLE_DEVICES=0 accelerate launch \
  --num_processes 1 \
  --num_machines 1 \
  --main_process_port 29502 \
  /app/train_distributed.py \
    --mode baseline \
    --bf16 \
    --torch-compile \
    --batch-size 32 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 17:38:49,235 [INFO] __main__: ============================================================
2025-11-03 17:38:49,235 [INFO] __main__: Запуск обучения в режиме: baseline
2025-11-03 17:38:49,235 [INFO] __main__: ============================================================
2025-11-03 17:38:49,235 [INFO] __main__: World size: 1
2025-11-03 17:38:49,235 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0
2025-11-03 17:38:49,235 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 17:38:49,236 [INFO] __main__: Создание baseline setup (single GPU)
Training samples:   1940063
Validation samples: 5000
2025-11-03 17:39:25,437 [INFO] lib.modeling: Модель: 960,881,664 параметров, dtype=torch.bfloat16, ~1.

### DeepSpeed_1

In [ ]:
name = 'DeepSpeed_1'  # 35523 MiB

!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29503 \
  /app/train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 1 \
    --reduce-bucket-size 500000000 \
    --allgather-bucket-size 500000000 \
    --overlap-comm \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 17:41:17,720 [INFO] __main__: ============================================================
2025-11-03 17:41:17,720 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 17:41:17,720 [INFO] __main__: ============================================================
2025-11-03 17:41:17,720 [INFO] __main__: World size: 2
2025-11-03 17:41:17,720 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-03 17:41:17,720 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 17:41:17,721 [INFO] __main__: Создание DeepSpeed setup (stage 1)
2025-11-03 17:41:17,721 [INFO] __

### DeepSpeed_2

In [ ]:
name = 'DeepSpeed_2'  # 33135 MiB

!CUDA_VISIBLE_DEVICES=0,1 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29504 \
  /app/train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 2 \
    --reduce-bucket-size 500000000 \
    --allgather-bucket-size 500000000 \
    --overlap-comm \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 18:12:33,701 [INFO] __main__: ============================================================
2025-11-03 18:12:33,701 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 18:12:33,701 [INFO] __main__: ============================================================
2025-11-03 18:12:33,701 [INFO] __main__: World size: 2
2025-11-03 18:12:33,701 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-03 18:12:33,701 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 18:12:33,701 [INFO] __main__: Создание DeepSpeed setup (stage 2)
2025-11-03 18:12:33,701 [INFO] __

### DeepSpeed_3

In [ ]:
name = 'DeepSpeed_3'  # 36829 MiB

!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29505 \
  /app/train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 3 \
    --reduce-bucket-size 500000000 \
    --allgather-bucket-size 500000000 \
    --overlap-comm \
    --stage3-prefetch-bucket-size 500000000 \
    --stage3-param-persistence-threshold 100000 \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 18:15:35,846 [INFO] __main__: ============================================================
2025-11-03 18:15:35,846 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 18:15:35,846 [INFO] __main__: ============================================================
2025-11-03 18:15:35,846 [INFO] __main__: World size: 2
2025-11-03 18:15:35,846 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-03 18:15:35,846 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 18:15:35,846 [INFO] __main__: Создание DeepSpeed setup (stage 3)
2025-11-03 18:15:35,846 [INFO] __

### FSDP_full_shard

In [ ]:
name = 'FSDP_full_shard'  # 30117 MiB
# FULL_SHARD | SHARD_GRAD_OP | NO_SHARD | HYBRID_SHARD | HYBRID_SHARD_ZERO2
# "full_shard", "shard_grad_op", "no_shard", "hybrid_shard"
    # --fsdp-cpu-offload \ выгрузка параметров на CPU (Уменьшает память GPU, но замедляет обучение)
    # --fsdp-activation-checkpointing \  Экономит память, но замедляет (~30% накладных) (Пересчитывает активации вместо сохранения)

!CUDA_VISIBLE_DEVICES=0,1 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 19:17:16,464 [INFO] __main__: ============================================================
2025-11-03 19:17:16,464 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:17:16,464 [INFO] __main__: ============================================================
2025-11-03 19:17:16,464 [INFO] __main__: World size: 2
2025-11-03 19:17:16,465 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-03 19:17:16,465 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 19:17:16,465 [INFO] __main__: Создание FSDP setup
2025-11-03 19:17:16,465 [INFO] __main__:   sharding_s

### FSDP_full_shard_v2

In [ ]:
name = 'FSDP_full_shard_v2'  # 30117 MiB

!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29502 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 16 \
    --grad-accum 2 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 19:37:23,097 [INFO] __main__: ============================================================
2025-11-03 19:37:23,097 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:37:23,097 [INFO] __main__: ============================================================
2025-11-03 19:37:23,190 [INFO] __main__: ============================================================
2025-11-03 19:37:23,190 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:37:23,190 [INFO] __main__: ============================================================
2025-11-03 19:37:23,190 [INFO] __

### FSDP_full_shard_v3

In [ ]:
name = 'FSDP_full_shard_v3'  # 20817 MiB

!CUDA_VISIBLE_DEVICES=0,1 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 8 \
    --grad-accum 4 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 19:51:37,530 [INFO] __main__: ============================================================
2025-11-03 19:51:37,530 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:51:37,530 [INFO] __main__: ============================================================
2025-11-03 19:51:37,530 [INFO] __main__: World size: 2
2025-11-03 19:51:37,530 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-03 19:51:37,530 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 19:51:37,530 [INFO] __main__: Создание FSDP setup
2025-11-03 19:51:37,530 [INFO] __main__:   sharding_s

### FSDP_full_shard_v4

In [ ]:
!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
    --num_processes 2 \
    --num_machines 1 \
    --mixed_precision bf16 \
    --main_process_port 29501 \
    ../train_distributed.py \
        --mode deepspeed \
        --deepspeed-stage 1 \
        --batch-size 16 \
        --grad-accum 4 \
        --run-name "ds_stage1"

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 13:23:21,061 [INFO] __main__: ============================================================
2025-11-03 13:23:21,061 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 13:23:21,061 [INFO] __main__: ============================================================
2025-11-03 13:23:21,061 [INFO] __main__: World size: 2
2025-11-03 13:23:21,061 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-03 13:23:21,062 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 13:23:21,062 [INFO] __main__: ============================================================
2025-11-03 13:23:21,062 [INFO] __main__: Запуск обучения 

In [19]:
name = "Baseline_1_gpu"

!CUDA_VISIBLE_DEVICES=1 accelerate launch \
    --main_process_port 29501 \
    --num_processes 1 \
    --num_machines 1 \
    /app/train_distributed.py \
        --mode baseline \
        --bf16 true \
        --batch-size 16 \
        --grad-accum 4 \
        --run-name {name} \
    2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 15:59:41,682 [INFO] __main__: ============================================================
2025-11-03 15:59:41,683 [INFO] __main__: Запуск обучения в режиме: baseline
2025-11-03 15:59:41,683 [INFO] __main__: ============================================================
2025-11-03 15:59:41,683 [INFO] __main__: World size: 1
2025-11-03 15:59:41,683 [INFO] __main__: CUDA_VISIBLE_DEVICES: 1
2025-11-03 15:59:41,683 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
False
2025-11-03 15:59:41,683 [INFO] __main__: Создание baseline setup (single GPU)
Training samples:   1940063
Validation samples: 5000
2025-11-03 16:00:16,922 [INFO] lib.modeling: Инициализирована модель: 960,881,664 параметров, dt

In [8]:
# Запуск на одном GPU без распределения
!CUDA_VISIBLE_DEVICES=3 python ../train_distributed.py \
    --mode baseline \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "baseline_1gpu"

python: can't open file '/app/hw2_parallel_pretrain/train_distributed.py': [Errno 2] No such file or directory


In [13]:
!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
    --num_processes 2 \
    --num_machines 1 \
    --mixed_precision bf16 \
    --main_process_port 29501 \
    ../train_distributed.py \
        --mode deepspeed \
        --deepspeed-stage 1 \
        --batch-size 16 \
        --grad-accum 4 \
        --run-name "ds_stage1"

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 13:23:21,061 [INFO] __main__: ============================================================
2025-11-03 13:23:21,061 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 13:23:21,061 [INFO] __main__: ============================================================
2025-11-03 13:23:21,061 [INFO] __main__: World size: 2
2025-11-03 13:23:21,061 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-03 13:23:21,062 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 13:23:21,062 [INFO] __main__: ============================================================
2025-11-03 13:23:21,062 [INFO] __main__: Запуск обучения 

In [ ]:
!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
    --num_processes 2 \
    --num_machines 1 \
    --mixed_precision bf16 \
    --main_process_port 29501 \
    ../train_distributed.py \
        --mode deepspeed \
        --deepspeed-stage 1 \
        --batch-size 16 \
        --grad-accum 4 \
        --run-name "ds_stage1"

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 13:16:38,590 [INFO] __main__: ============================================================
2025-11-03 13:16:38,590 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 13:16:38,591 [INFO] __main__: ============================================================
2025-11-03 13:16:38,669 [INFO] __main__: ============================================================
2025-11-03 13:16:38,669 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 13:16:38,669 [INFO] __main__: ============================================================
2025-11-03 13:16:38,669 [INFO] __main__: World size: 2
2025-11-03 13:16:3

### DeepSpeed (stage 1) baseline
- GPU utilization: 58057 MiB

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 accelerate launch \
    --num_processes 2 \
    --num_machines 1 \
    --mixed_precision bf16 \
    ../train_distributed.py \
        --batch-size 2 \
        --grad-accum 8 \
        --mode deepspeed \
        --deepspeed-stage 1 \
        --run-name "DeepSpeed_1"

In [1]:
!CUDA_VISIBLE_DEVICES=0,1 deepspeed --num_gpus=2 ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 1 \
    --run-name "DeepSpeed_1"

[2025-11-03 08:00:43,127] [WARNING] [runner.py:232:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected VISIBLE_DEVICES=0,1 but ignoring it because one or several of --include/--exclude/--num_gpus/--num_nodes cl args were used. If you want to use CUDA_VISIBLE_DEVICES don't pass any of these arguments to deepspeed.
[2025-11-03 08:00:43,127] [INFO] [runner.py:630:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMV19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None --log_level=info ../train_distributed.py --mode deepspeed --deepspeed-stage 1 --run-name DeepSpeed_1
[2025-11-03 08:00:48,993] [INFO] [launch.py:155:main] 0 NCCL_VERSION=2.23.4
[2025-11-03 08:00:48,993] [INFO] [launch.py:155:main] 0 AWS_OFI_NCCL_VERSION=1.12.1
[2025-11-03 08:00:48,994] [INFO] [launch.py:162:main] WORLD INFO DICT: {'localhost': [0, 1]}
[2025-11-03 08:00:48,994] [INFO] [launch.py:168:main] nn

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 deepspeed --num_gpus=2 ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 2 \
    --run-name "DeepSpeed_1"

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 deepspeed --num_gpus=2 ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 3 \
    --run-name "DeepSpeed_1"

In [20]:
!accelerate launch \
  --num_processes 2 --num_machines 1 --gpu_ids 2,3 \
  --mixed_precision bf16 \
  ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 1 \
    --run-name "DeepSpeed_1"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 07:26:22,810 [INFO] __main__: ============================================================
2025-11-03 07:26:22,810 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 07:26:22,810 [INFO] __main__: ============================================================
2025-11-03 07:26:22,810 [INFO] __main__: World size: 1
2025-11-03 07:26:22,810 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-03 07:26:22,810 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 07:26:22,812 [INFO] __main__: Создание DeepSpeed setup (stage 1)
Loading dataset shards: 100%|█████████████████| 27/27 [00:00<00:00, 4215.85it/s]
Training samples:   1940063
Validation samples: 5000
2025-11-03 07:26:58,404 [INFO] lib.modeling: Инициализирована модель: 

In [14]:
!CUDA_VISIBLE_DEVICES=1 torchrun --nproc_per_node=1 --master_port=29501 ../train_distributed.py \
    --mode baseline \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "baseline_test"

# ~ 29.5гб на 1 GPU, загрузка полная (95-100%)
# Успело пройти 1175 шагов обучения

Traceback (most recent call last):
  File "/app/hw2_parallel_pretrain/../train_distributed.py", line 9, in <module>
    from lib.constants import OUTPUT_DIR
  File "/app/lib/__init__.py", line 24, in <module>
    from .tokenizer import build_tokenizer
  File "/app/lib/tokenizer.py", line 7, in <module>
    from transformers import AutoTokenizer, PreTrainedTokenizerBase
  File "/usr/local/lib/python3.12/dist-packages/transformers/__init__.py", line 1018, in <module>
    import_structure = define_import_structure(Path(__file__).parent / "models", prefix="models")
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2597, in define_import_structure
W1103 07:19:13.956000 817813 torch/distributed/elastic/agent/server/api.py:719] Received 2 death signal, shutting down workers
    import_structure = create_import_structure_from_path(module_path)
              

___

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 torchrun --nproc_per_node=2 --master_port=29502 ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 1 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "deep_speed_setup_1"

# ~ по 53гб на каждый GPU, загрузка полная (95-100%)

2025-11-02 18:36:29,438 [INFO] __main__: ============================================================
2025-11-02 18:36:29,438 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:36:29,438 [INFO] __main__: ============================================================
2025-11-02 18:36:29,879 [INFO] __main__: ============================================================
2025-11-02 18:36:29,879 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:36:29,879 [INFO] __main__: ============================================================
2025-11-02 18:36:29,879 [INFO] __main__: World size: 2
2025-11-02 18:36:29,879 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-02 18:36:29,879 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-02 18:36:29,879 [INFO] __main__: Создание DeepSpeed setup (stage 1)
Loading dataset shards: 100%|█████████████████| 27/27 [00:00<00:00, 2788.49it/s]
Training samples:   1940063
Validation samples: 5000
Loading dataset shards: 100%|████

___

In [2]:
!CUDA_VISIBLE_DEVICES=2,3 torchrun --nproc_per_node=2 --master_port=29501 ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 2 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "deep_speed_setup_2"

# ~ по 50гб на каждый GPU, загрузка полная (95-100%)

2025-11-02 18:35:36,764 [INFO] __main__: ============================================================
2025-11-02 18:35:36,764 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:35:36,764 [INFO] __main__: ============================================================
2025-11-02 18:35:36,764 [INFO] __main__: World size: 2
2025-11-02 18:35:36,764 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-02 18:35:36,764 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-02 18:35:36,764 [INFO] __main__: Создание DeepSpeed setup (stage 2)
2025-11-02 18:35:37,066 [INFO] __main__: ============================================================
2025-11-02 18:35:37,066 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:35:37,067 [INFO] __main__: ============================================================
Loading dataset shards: 100%|█████████████████| 27/27 [00:00<00:00, 2488.00it/s]
Training samples:   1940063
Validation samples: 5000
Loading dataset shards: 100%|████

___

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 torchrun --nproc_per_node=2 ../train_distributed.py \
    --mode fsdp \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "fsdp"